In [ ]:

from scipy import stats

import matplotlib.pyplot as plt
import numpy as np

import numpy.random as rd

import pandas as pd 

from tqdm import tqdm
import math

import autograd.numpy as np # Numpy用の薄いラッパ
from autograd import grad

In [ ]:
# parameter
mus = 5
sigmas = 1

In [ ]:
# うまく行った場合の正規分布の乱数

arr = np.random.normal(mus,sigmas, 10)
print(arr)

# 確率分布の定義

In [ ]:
# 遅いバージョン
"""
def h(x, mu, sigma):
    return -1 * np.log(stats.norm.pdf(x=x, loc=mu, scale=sigma) )
    
"""

In [ ]:
# 遅いバージョン
"""
def delta_h(x, mu, sigma):
    return  -1*stats.norm.pdf(x=x, loc=mu, scale=sigma)*(x-mus)/ (sigma )
"""

In [ ]:
import autograd.numpy as np  # Thinly-wrapped numpy
from autograd import grad    # The only autograd function you may ever need

def func(x,y):   
    # Define a function
    return x**2 + y*x

grad_tanh = grad(func)
             
grad_tanh(1.0,1)

In [ ]:
def h(x, mu, sigma):
    return (x-mu)**2/2 * sigma

In [ ]:
def grad_func(h):
    return grad(h)

In [ ]:
def delta_h(x, mu, sigma):
    grad_func = grad(h)
    return grad_func(float(x), float(mu), float(sigma))
    

In [ ]:
def hamiltonian(x,mu, sigma,p):
    return h(x, mu, sigma) + 0.5 * p**2

In [ ]:
def leap_flog_step1(p, eta,x,mu, sigma):
    return p - 0.5*eta* delta_h(x, mu, sigma)

def leap_flog_step2(p, x, eta):
    return x + eta * p


In [ ]:
def move_one_step(x,mu, sigma, p, eta, L=100, stlide=1):
    """
    リープフロッグ法でL回移動した１ステップを実行
    """
    ret = []
    ret.append([1, p, x, hamiltonian(x,mu, sigma,p)])
    for _ in range(L):
        p = leap_flog_step1(p, eta,x,mu, sigma)
        x = leap_flog_step2(p, x, eta)
        p = leap_flog_step1(p, eta,x,mu, sigma)
        ret.append([1, p, x, hamiltonian(x,mu, sigma,p)])
    return ret[::stlide]

In [ ]:

# initial param
x = 3

p = 1
eta = 0.01
L = 100

result = move_one_step(x,mus, sigmas, p, eta, L, stlide=1)

In [ ]:
result = np.array(result)
#HTML(pd.DataFrame(result, columns="p,x,hamiltonian".split(",")).to_html())
# type=1はリープフロッグ法による遷移を表す
pd.DataFrame(result, columns="type,p,x,hamiltonian".split(","))

In [ ]:
# HMC simulation
rd.seed(71)
scale_p = 1

# initial param
x = 2.5
p = rd.normal(loc=0,scale=scale_p)

T = 5000


sim_result = []
prev_hamiltonian = hamiltonian(x,mus, sigmas,p)
sim_result.append([ p, x, prev_hamiltonian, True])
for t in tqdm(range(T)):
    prev_p = p
    prev_x = x
    prev_hamiltonian = hamiltonian(x,mus, sigmas,p)
    for i in range(L):
        p = leap_flog_step1(p, eta,x,mus, sigmas)
        x = leap_flog_step2(p, x, eta)
        p = leap_flog_step1(p, eta,x,mus, sigmas)
        

    H = hamiltonian(x,mus, sigmas,p)
    r = np.exp(prev_hamiltonian-H)
    if  r > 1:
        sim_result.append([ p, x, hamiltonian(x,mus, sigmas,p), True])
    elif r > 0 and rd.uniform() < r:
        sim_result.append([ p, x,hamiltonian(x,mus, sigmas,p), True])
    else:

        sim_result.append([ p, x, hamiltonian(x,mus, sigmas,p), False])
        x = prev_x
    
    p = rd.normal(loc=0,scale=scale_p)
    
sim_result = np.array(sim_result)
df = pd.DataFrame(sim_result, columns="p,x,hamiltonian,accept".split(","))

In [ ]:
sum(df["accept"])/df["accept"].shape[0]

In [ ]:
df

In [ ]:
burn_in = int(T*0.1)
w = 0.05
n = len(sim_result[burn_in:])


plt.figure(figsize=(14,8))
plt.hist(sim_result[burn_in:,1])
